Transformers를 사용한 한국어 wav2vec2 활용에 대한 문서입니다.

test는 zeroth_korean의 testset을 사용했습니다.

TF가 좋지만 다들 torch로 해서 어쩔수없이 torch로 합니다.

# 1. 설치

pip를 통해 설치를 진행합니다.


In [ ]:
!pip install transformers
!pip install datasets
!pip install jiwer

# 2. 라이브러리 불러오기

wav2vec2 기반이라서 별도의 feature extraction을 필요로 하지 않습니다.

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset
import soundfile as sf
import torch
from jiwer import wer


In [7]:
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')

ds = load_dataset("kresnik/zeroth_korean", "clean")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Computing checksums: 100%|##########| 1/1 [00:30<00:00, 30.19s/it]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset zeroth_korean downloaded and prepared to /root/.cache/huggingface/datasets/kresnik___zeroth_korean/clean/1.0.1/e5d146fc495c84b4b1471118f43a266048059e6a0ccd6c0e23b34322b1d6d379. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

# 3. 전체 데이터 중 Test set만을 이용함

파일 경로를 읽어서 raw signal을 저장합니다.

이어서 전체 test set에 대하여 결과를 result['transcription']에 저장합니다.

기대되는 WER은 4.74% (0.0474) 입니다.

In [9]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 22263
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 457
    })
})


In [10]:
test_ds = ds['test']
print(test_ds)
print(test_ds[0])

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 457
})
{'file': '/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/test_data_01/003/105/105_003_0029.flac', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/test_data_01/003/105/105_003_0029.flac', 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00012207,
        0.00018311, -0.00012207], dtype=float32), 'sampling_rate': 16000}, 'text': '몬터규는 자녀들이 사랑을 제대로 못 받고 크면 매우 심각한 결과가 초래된다는 결론을 내렸습니다', 'speaker_id': 105, 'chapter_id': 3, 'id': '105_003_0029'}


In [11]:
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

test_ds = test_ds.map(map_to_array)

  0%|          | 0/457 [00:00<?, ?ex/s]

In [12]:
print(test_ds[0].keys())
print(test_ds[0]['speech'][:20])

dict_keys(['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id', 'speech'])
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.0517578125e-05, 0.0, 0.0, 0.0, 0.0, -3.0517578125e-05, -3.0517578125e-05, -3.0517578125e-05, -3.0517578125e-05, -6.103515625e-05, -3.0517578125e-05, 0.0]


In [13]:
test_ds[0]["file"]

'/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/test_data_01/003/105/105_003_0029.flac'

In [14]:
print(test_ds[0]["audio"]["array"][:20])
print(test_ds[0]['speech'][:20])

[ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
 -3.0517578e-05  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -3.0517578e-05 -3.0517578e-05 -3.0517578e-05
 -3.0517578e-05 -6.1035156e-05 -3.0517578e-05  0.0000000e+00]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.0517578125e-05, 0.0, 0.0, 0.0, 0.0, -3.0517578125e-05, -3.0517578125e-05, -3.0517578125e-05, -3.0517578125e-05, -6.103515625e-05, -3.0517578125e-05, 0.0]


In [15]:
def map_to_pred(batch):
    inputs = processor(batch["speech"], sampling_rate=16000, return_tensors="pt", padding="longest")
    input_values = inputs.input_values.to("cuda")
    #attention_mask = inputs.attention_mask.to("cuda")
    
    with torch.no_grad():
        #logits = model(input_values, attention_mask=attention_mask).logits
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

result = test_ds.map(map_to_pred, batched=True, batch_size=16, remove_columns=["speech"])

  0%|          | 0/29 [00:00<?, ?ba/s]

In [16]:
print("WER:", wer(result["text"], result["transcription"]))

WER: 0.048034934497816595


In [17]:
index=4
print(result["text"][index])
print(result["transcription"][index])
from IPython.display import Audio
Audio(test_ds[index]["audio"]["array"], rate=16000)

평소 오전 아홉 시 에서 오후 일곱 시까지 일하면 하루 이 만원 정도를 번다
평소 오전 아홉 시 서 오후 일곱 시까지 일하면 하루 이 만원 정도를 번다


In [81]:
index=6
inputs = processor(test_ds[index]["audio"]["array"], sampling_rate=16000, return_tensors="pt", padding="longest")
input_values = inputs.input_values.to("cuda")

with torch.no_grad():
    logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

print(test_ds[index]["text"])
print(transcription[0])
from IPython.display import Audio
Audio(test_ds[index]["audio"]["array"], rate=16000)

야권은 여당에서 분명한 입장을 표해야 한다고 연일 압박 수위를 높이고 있다
야권은 여당에서 분명한 입장을 표해야 한다고 연일 압박 수위를 높이고 있다


In [82]:
train_ds = ds['train']

In [83]:
index=2
inputs = processor(train_ds[index]["audio"]["array"], sampling_rate=16000, return_tensors="pt", padding="longest")
input_values = inputs.input_values.to("cuda")

with torch.no_grad():
    logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

print(train_ds[index]["text"])
print(transcription[0])
from IPython.display import Audio
Audio(train_ds[index]["audio"]["array"], rate=16000)

차 문이 종잇장처럼 얇지 않으니 문 두께 스물 센티미터를 빼면 실제 승 하차 여유 공간은 스물 센티미터라는 계산이 나옵니다
차 문이 종잇장처럼 얇지 않으니 문 두께 스물 센티미터를 빼면 실제 승 하차 여유 공간은 스물 센티미터라는 계산이 나옵니다


## 다른 음성 파일 test

In [84]:
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
from IPython.display import Audio
import torchaudio
import torchaudio.transforms as T
import librosa

def  speech_recognition(filename):
    speech, sample_rate = torchaudio.load(filename)
    print(sample_rate)
    print(speech.shape)
    resample_rate = 16000
    if( sample_rate != resample_rate):
        resampler = T.Resample(sample_rate, resample_rate, dtype=speech[0].dtype)
        resampled_waveform = resampler(speech[0])
    inputs = processor(resampled_waveform, sampling_rate=resample_rate, return_tensors="pt", padding="longest")
    input_values = inputs.input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)

    print(transcription[0])
    

In [96]:
filename = "/content/drive/MyDrive/AI_sound/5.example/test4.wav"
speech_recognition(filename)
Audio(filename)

44100
torch.Size([1, 226541])
남바직전에 새정치연합을 책임지게 된 문의상 비대위원장이 이런 말을 였습니다


## wav file에서 channel을 1개로 줄이기

In [89]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [94]:
from pydub import AudioSegment
sound = AudioSegment.from_wav("/content/drive/MyDrive/AI_sound/5.example/test4.wav")
sound = sound.set_channels(1)
sound.export("/content/drive/MyDrive/AI_sound/5.example/test4.wav", format="wav")

<_io.BufferedRandom name='/content/drive/MyDrive/AI_sound/5.example/test4.wav'>